In [3]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import pandas as pd

In [22]:
path='../data/2city11.txt'

In [49]:
#text_df=pd.read_table('../data/2city11.txt')

# Open a file: file
file = open('../article.txt',mode='r')
 
# read all lines at once
all_of_it = file.read()
 
# close the file
file.close()

In [50]:
all_of_it

'In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses. The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming lives and industry 

In [15]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []
    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        sentences.pop() 
    
    return sentences

In [16]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

In [18]:
def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    # Step 1 - Read text and tokenize
    sentences =  read_article(file_name)
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    # Step 5 - Offcourse, output the summarize texr
        print("Summarize Text: \n", ". ".join(summarize_text))

In [4]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
 
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

# let's begin
generate_summary('../article.txt', 2)

In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills
Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services
As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses
The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming lives and industry and 

In [51]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
#str_abstract=[str(abstract) for abstract in df['title']]
#abstract_array = [clean(abstract).split() for abstract in str_abstract] 

In [52]:
sents=[clean(i) for i in all_of_it.split('.')]

In [53]:
sents

['attempt build aiready workforce microsoft announced intelligent cloud hub launched empower next generation student aiready skill',
 'envisioned threeyear collaborative program intelligent cloud hub support around 100 institution ai infrastructure course content curriculum developer support development tool give student access cloud ai service',
 'part program redmond giant want expand reach planning build strong developer ecosystem india program set core ai infrastructure iot hub selected campus',
 'company provide ai development tool azure ai service microsoft cognitive service bot service azure machine learning',
 'according manish prakash country general managerps health education microsoft india said with ai defining technology time transforming life industry job tomorrow require different skillset',
 'require collaboration training working ai',
 'thatâ€™s become critical ever educational institution integrate new cloud ai technology',
 'program attempt ramp institutional setup b

In [67]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    print(sent1)
    print(sent2)
    all_words = list(set(sent1 + sent2))
    print(all_words)
    print(sent1 + sent2)
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    print(vector1)
    print(vector2)
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
        print(all_words.index(w))
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
        print(all_words.index(w))
    print(vector1)
    print(vector2)
    return 1 - cosine_distance(vector1, vector2)

In [68]:
sentence_similarity(sents[1],sents[2])

['e', 'n', 'v', 'i', 's', 'i', 'o', 'n', 'e', 'd', ' ', 't', 'h', 'r', 'e', 'e', 'y', 'e', 'a', 'r', ' ', 'c', 'o', 'l', 'l', 'a', 'b', 'o', 'r', 'a', 't', 'i', 'v', 'e', ' ', 'p', 'r', 'o', 'g', 'r', 'a', 'm', ' ', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 't', ' ', 'c', 'l', 'o', 'u', 'd', ' ', 'h', 'u', 'b', ' ', 's', 'u', 'p', 'p', 'o', 'r', 't', ' ', 'a', 'r', 'o', 'u', 'n', 'd', ' ', '1', '0', '0', ' ', 'i', 'n', 's', 't', 'i', 't', 'u', 't', 'i', 'o', 'n', ' ', 'a', 'i', ' ', 'i', 'n', 'f', 'r', 'a', 's', 't', 'r', 'u', 'c', 't', 'u', 'r', 'e', ' ', 'c', 'o', 'u', 'r', 's', 'e', ' ', 'c', 'o', 'n', 't', 'e', 'n', 't', ' ', 'c', 'u', 'r', 'r', 'i', 'c', 'u', 'l', 'u', 'm', ' ', 'd', 'e', 'v', 'e', 'l', 'o', 'p', 'e', 'r', ' ', 's', 'u', 'p', 'p', 'o', 'r', 't', ' ', 'd', 'e', 'v', 'e', 'l', 'o', 'p', 'm', 'e', 'n', 't', ' ', 't', 'o', 'o', 'l', ' ', 'g', 'i', 'v', 'e', ' ', 's', 't', 'u', 'd', 'e', 'n', 't', ' ', 'a', 'c', 'c', 'e', 's', 's', ' ', 'c', 'l', 'o', 'u', 'd',

0.9524883259067902

In [40]:
import numpy as np
from scipy.sparse import csc_matrix

def pageRank(G, s = .85, maxerr = .0001):
    """
    Computes the pagerank for each of the n states
    Parameters
    ----------
    G: matrix representing state transitions
       Gij is a binary value representing a transition from state i to j.
    s: probability of following a transition. 1-s probability of teleporting
       to another state.
    maxerr: if the sum of pageranks between iterations is bellow this we will
            have converged.
    """
    n = G.shape[0]

    # transform G into markov matrix A
    A = csc_matrix(G,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    # bool array of sink states
    sink = rsums==0

    # Compute pagerank r until we converge
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
        # calculate each pagerank at a time
        for i in range(0,n):
            print(n)
            # inlinks of state i
            Ai = np.array(A[:,i].todense())[:,0]
            print(Ai)
            # account for sink states
            Di = sink / float(n)
            print(Di)
            # account for teleportation to state i
            Ei = np.ones(n) / float(n)
            print(Ei)
            r[i] = ro.dot( Ai*s + Di*s + Ei*(1-s) )
            print(r[i])
    # return normalized pagerank
    return r/float(sum(r))




if __name__=='__main__':
    # Example extracted from 'Introduction to Information Retrieval'
    G = np.array([[0,0,1,0,0,0,0],
                  [0,1,1,0,0,0,0],
                  [1,0,1,1,0,0,0],
                  [0,0,0,1,1,0,0],
                  [0,0,0,0,0,0,1],
                  [0,0,0,0,0,1,1],
                  [0,0,0,1,1,0,1]])
pageRank(G,s=.86)

7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
1.0
7
[0.  0.5 0.  0.  0.  0.  0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.5700000000000001
7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
1.1433333333333333
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
1.2866666666666666
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
1.4300000000000002
7
[0.  0.  0.  0.  0.  0.5 0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.5700000000000001
7
[0.         

1.6525343504514525
7
[0.  0.  0.  0.  0.  0.5 0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.20676740976250016
7
[0.         0.         0.         0.         0.33333333 0.33333333
 0.33333333]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.9289907939541429
7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.7173666872970004
7
[0.  0.5 0.  0.  0.  0.  0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.20328492899792697
7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.6887087056851475
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14

7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.504350127405787
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.9329741946906887
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
1.1940222731553998
7
[0.  0.  0.  0.  0.  0.5 0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.1492595444690974
7
[0.         0.         0.         0.         0.33333333 0.33333333
 0.33333333]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.6715732368221785
7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14

0.7208619131496427
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.922562701009514
7
[0.  0.  0.  0.  0.  0.5 0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.11532520834305698
7
[0.         0.         0.         0.         0.33333333 0.33333333
 0.33333333]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.5188927141924553
7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.3988948431942653
7
[0.  0.5 0.  0.  0.  0.  0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.1133590808833296
7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.1

0.08910564779082071
7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.301085667892192
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.5569716211617892
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.7128150997648547
7
[0.  0.  0.  0.  0.  0.5 0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.08910564779082071
7
[0.         0.         0.         0.         0.33333333 0.33333333
 0.33333333]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.4009207919539606
7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.142

...

 0.33333333]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0026449744605606756
7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0020333037414158412
7
[0.  0.5 0.  0.  0.  0.  0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.000577830151281268
7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0019524730596207577
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0036118361082973895
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.004622446133340943
7


 0.33333333]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.001940879836179609
7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.001492036422803877
7
[0.  0.5 0.  0.  0.  0.  0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.00042401123567775137
7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0014327229425491477
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.002650362027576569
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0033919467381647445
7


[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0011138443995575272
7
[0.  0.5 0.  0.  0.  0.  0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0003165355302262685
7
[0.5        0.33333333 0.33333333 0.         0.         0.
 0.        ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0010695653278202282
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0019785649037095494
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0025321773786207637
7
[0.  0.  0.  0.  0.  0.5 0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.000

0.0007848460577585578
7
[0.         0.         0.33333333 0.5        0.         0.
 0.5       ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0014518691138395668
7
[0.  0.  0.  1.  0.  0.  0.5]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.001858109440782042
7
[0.  0.  0.  0.  0.  0.5 0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.00023227348211156477
7
[0.         0.         0.         0.         0.33333333 0.33333333
 0.33333333]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0010450882829737124
7
[0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
0.0008034035668646336
7
[0.  0.5 0.  0.  0.  0.  0. ]
[0. 0. 0. 0. 0. 0. 0.]
[0.14285714 0.14285714 0.14285714 0.14285714 0.142857

array([0.12727557, 0.03616954, 0.12221594, 0.22608452, 0.28934412,
       0.03616954, 0.16274076])

In [42]:
G = np.array([[0,0,1,0,0,0,0],
                  [0,1,1,0,0,0,0],
                  [1,0,1,1,0,0,0],
                  [0,0,0,1,1,0,0],
                  [0,0,0,0,0,0,1],
                  [0,0,0,0,0,1,1],
                  [0,0,0,1,1,0,1]])

In [44]:
import numpy as np
from scipy.sparse import csc_matrix
A = csc_matrix(G,dtype=np.float)
print(A)
rsums=np.array(A.sum(1))[:,0]
print(rsums)
ri, ci = A.nonzero()
print(ri,ci)
A.data /= rsums[ri]
print(A)

  (2, 0)	1.0
  (1, 1)	1.0
  (0, 2)	1.0
  (1, 2)	1.0
  (2, 2)	1.0
  (2, 3)	1.0
  (3, 3)	1.0
  (6, 3)	1.0
  (3, 4)	1.0
  (6, 4)	1.0
  (5, 5)	1.0
  (4, 6)	1.0
  (5, 6)	1.0
  (6, 6)	1.0
[1. 2. 3. 2. 1. 2. 3.]
[0 1 1 2 2 2 3 3 4 5 5 6 6 6] [2 1 2 0 2 3 3 4 6 5 6 3 4 6]
  (2, 0)	1.0
  (1, 1)	0.5
  (0, 2)	0.5
  (1, 2)	0.3333333333333333
  (2, 2)	0.3333333333333333
  (2, 3)	0.3333333333333333
  (3, 3)	0.5
  (6, 3)	0.5
  (3, 4)	1.0
  (6, 4)	0.5
  (5, 5)	0.5
  (4, 6)	0.3333333333333333
  (5, 6)	0.3333333333333333
  (6, 6)	0.3333333333333333


In [32]:
rsums=np.array(A.sum(1))
rsums

array([[1.],
       [2.],
       [3.],
       [2.],
       [1.],
       [2.],
       [3.]])

In [23]:
A.data

array([1.        , 0.5       , 0.5       , 0.33333333, 0.33333333,
       0.33333333, 0.5       , 0.5       , 1.        , 0.5       ,
       0.5       , 0.33333333, 0.33333333, 0.33333333])

In [24]:
A = csc_matrix(G,dtype=np.float)

In [25]:
A.data

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [26]:
ri

array([0, 1, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 6], dtype=int32)

In [27]:
ci

array([2, 1, 2, 0, 2, 3, 3, 4, 6, 5, 6, 3, 4, 6], dtype=int32)